# Bien penser à changer le CSV et le nom des DF !!

In [2]:
from selenium import webdriver
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import re, time, random

In [4]:
df_data_villes_clean5 = pd.read_csv("pays_de_la_loire5.csv", 
                                    delimiter=",",
                                    dtype=str,
                                    index_col=0)


In [ ]:
df_data_villes_clean5.head()


,#Code_commune_INSEE,Lib_MA,departement,Code_postal,Nom_de_la_commune_url,url
31714,72146,GUECELARD,72,72230,guecelard,https://www.meilleursagents.com/prix-immobilie...
31715,72147,LA GUIERCHE,72,72380,la-guierche,https://www.meilleursagents.com/prix-immobilie...
31716,72148,JAUZE,72,72110,jauze,https://www.meilleursagents.com/prix-immobilie...
31717,72149,JOUE EN CHARNIE,72,72540,joue-en-charnie,https://www.meilleursagents.com/prix-immobilie...
31718,72150,JOUE L ABBE,72,72380,joue-l-abbe,https://www.meilleursagents.com/prix-immobilie...


In [28]:
def extract_number(text):
    """Nettoyer le texte et transformer en float"""
    clean_text = text.replace("\u202f", "").replace("\xa0", "").replace("€", "").strip()
    clean_text = clean_text.replace(",", ".")
    return float(clean_text)

# Selenium setup (Chrome auto-managed by Selenium Manager; Chrome must be installed)
options = webdriver.ChromeOptions()
options.add_argument("--lang=fr-FR")
options.add_argument("--disable-blink-features=AutomationControlled")
# Uncomment to persist cookies/session between runs:
# options.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Local\Google\Chrome\User Data\ScrapeProfile")

driver = webdriver.Chrome(options=options)
stealth(driver,
    languages=["fr-FR", "fr"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)
driver.set_window_size(1366, 768)

# Lancer le navigateur
driver.get("https://www.meilleursagents.com/prix-immobilier/")

In [29]:


# Liste pour stocker toutes les données
all_data = []
error_data = []

for index, row in df_data_villes_clean5.iterrows():
    url = row['url']
    code_insee = row['#Code_commune_INSEE']
    try:
        # Human-like pacing
        time.sleep(random.uniform(1.8, 3.6))

        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        data_dict = {}
        # 1-Ajouter la ville
        title_text = soup.find("title").get_text().strip()
        ville = title_text.split("Prix immobilier")[1].split("(")[0].strip()
        data_dict["ville"] = ville
        
        #2-ajouter prix achat+loyer au m2 appt et maison
        raw_text3 = soup.find_all("ul", class_="prices-summary__price-range")
        for ul in raw_text3:
            li_items = ul.find_all("li")
            label = li_items[0].get_text().strip().lower()
            main_value = extract_number(li_items[1].get_text())
            # Nettoyer le texte pour récupérer min et max correctement
            range_text = li_items[2].get_text()
            range_text = range_text.replace("\u202f", "").replace("\xa0", "").replace(",", ".")
            # Extraire tous les nombres sous forme de float
            numbers_in_range = [float(n) for n in re.findall(r"[\d\.]+", range_text)]
            if "prix" in label:
                if "prix_appartement" not in data_dict:
                    data_dict["prix_appartement"] = main_value
                    data_dict["min_appartement"] = numbers_in_range[0]
                    data_dict["max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["prix_maison"] = main_value
                    data_dict["min_maison"] = numbers_in_range[0]
                    data_dict["max_maison"] = numbers_in_range[1]
            elif "loyer" in label:
                if "loyer_appartement" not in data_dict:
                    data_dict["loyer_appartement"] = main_value
                    data_dict["loyer_min_appartement"] = numbers_in_range[0]
                    data_dict["loyer_max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["loyer_maison"] = main_value
                    data_dict["loyer_min_maison"] = numbers_in_range[0]
                    data_dict["loyer_max_maison"] = numbers_in_range[1]
        
        #3-Ajouter l'URL pour référence
        data_dict["url"] = url
        data_dict['Code_insee'] = code_insee
        all_data.append(data_dict)
        print(f"Datas ajoutées pour la ville : {ville}")
        
    except Exception as erreur:
        print(f"Erreur pour l'URL {url}: {erreur}")
        error_data.append({
        "url": url,
        "erreur": str(erreur)})
        
    # Pause pour ne pas surcharger le site
    time.sleep(random.uniform(2.3, 3.6))

driver.quit()



Datas ajoutées pour la ville : Guécélard
Datas ajoutées pour la ville : La Guierche
Datas ajoutées pour la ville : Jauzé


KeyboardInterrupt: 

In [30]:
df_error_url = pd.DataFrame(error_data)
df_meilleur_agent5 = pd.DataFrame(all_data)

In [31]:
df_meilleur_agent5.head()

,ville,prix_appartement,min_appartement,max_appartement,prix_maison,min_maison,max_maison,loyer_appartement,loyer_min_appartement,loyer_max_appartement,loyer_maison,loyer_min_maison,loyer_max_maison,url,Code_insee
0,Guécélard,1579.0,1199.0,2027.0,1800.0,822.0,2849.0,11.3,7.7,14.9,11.2,8.7,14.7,https://www.meilleursagents.com/prix-immobilie...,72146
1,La Guierche,1732.0,1096.0,2527.0,1724.0,690.0,2538.0,10.3,8.5,13.5,10.7,9.1,12.9,https://www.meilleursagents.com/prix-immobilie...,72147
2,Jauzé,1625.0,1013.0,2343.0,1253.0,533.0,2506.0,11.7,7.9,19.2,10.1,7.4,13.4,https://www.meilleursagents.com/prix-immobilie...,72148


In [26]:
print(df_meilleur_agent5.shape)
print(df_duplicate.shape)

(262, 14)
(259, 14)


In [25]:
df_duplicate = df_meilleur_agent5.drop_duplicates(subset=['ville'])

In [ ]:
df_meilleur_agent5.to_csv('ma_11.csv',index=False)
df_error_url.to_csv('error_ma_11.csv',index=False)

In [47]:
df_meilleur_agent9['ratio_appt'] = round((df_meilleur_agent9['prix_appartement'] / df_meilleur_agent9['loyer_appartement']),1)

df_meilleur_agent9['ratio_maison'] = round((df_meilleur_agent9['prix_maison'] / df_meilleur_agent9['loyer_maison']),1)

df_meilleur_agent9 = df_meilleur_agent9.sort_values('ratio_appt')